In [8]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

import scipy.sparse as sparse

from implicit.als import AlternatingLeastSquares

In [10]:
df_user_taggedbookmarks = pd.read_table('user_taggedbookmarks.dat', engine='python', encoding='utf-8', sep='\t')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


In [11]:
df_user_taggedbookmarks.head()

,userID,bookmarkID,tagID,day,month,year,hour,minute,second
0,8,1,1,8,11,2010,23,29,22
1,8,2,1,8,11,2010,23,25,59
2,8,7,1,8,11,2010,18,55,1
3,8,7,6,8,11,2010,18,55,1
4,8,7,7,8,11,2010,18,55,1


In [12]:
len(df_user_taggedbookmarks)

437593

In [13]:
df = df_user_taggedbookmarks.drop(['tagID','day','month','year','hour','minute','second'], axis=1)

In [14]:
df.head()

,userID,bookmarkID
0,8,1
1,8,2
2,8,7
3,8,7
4,8,7


In [15]:
df = df.dropna()

In [16]:
len(df)

437593

In [17]:
df.columns = ['uid', 'iid']

In [18]:
df.head()

,uid,iid
0,8,1
1,8,2
2,8,7
3,8,7
4,8,7


In [19]:
# тут, наверное, лучше применить explicit с подсчетом тегов на bookmarkID плюс учесть их веса
# df.groupby(['uid', 'iid']).count()
df = df[['uid', 'iid']].drop_duplicates()

In [20]:
df.head()

,uid,iid
0,8,1
1,8,2
2,8,7
5,8,8
8,8,9


In [21]:
df.describe()

,uid,iid
count,104799.000000,104799.000000
mean,46948.284211,40919.907709
std,32649.253458,31540.365773
min,8.000000,1.000000
25%,14974.000000,13812.500000
50%,45151.000000,30734.000000
75%,75324.000000,66868.500000
max,108035.000000,107253.000000


In [22]:
# в матрицу свалила просто единички, это ничего?
# типа: такой-то юзер положил в закладки то-то bookmarkID
df['rate'] = 1

In [23]:
df.head()

,uid,iid,rate
0,8,1,1
1,8,2,1
2,8,7,1
5,8,8,1
8,8,9,1


In [24]:
userids = list(np.sort(df.uid.unique()))
itemids = list(np.sort(df.iid.unique()))
rates = list(df.rate)

In [26]:
len(userids)

1867

In [27]:
len(itemids)

69223

In [28]:
len(rates)

104799

In [29]:
rows = df.uid.astype(int)
cols = df.iid.astype(int)

In [30]:
# размерность shape=(len(itemids), len(userids)) не задала, потому что
rows.shape[0]

104799

In [31]:
# меньше, чем 
rows.max()

108035

In [32]:
df_sparse = sparse.csr_matrix((rates, (cols, rows)))

In [33]:
model50 = AlternatingLeastSquares(factors=50)
model50.fit(df_sparse)

item-to-user

In [34]:
uid = 8
items_to_user = df_sparse.T.tocsr()

In [35]:
recommendations50 = model50.recommend(uid, items_to_user)
recommendations50

[(2178, 0.10267255),
 (5874, 0.08410659),
 (5845, 0.08029371),
 (5824, 0.07581681),
 (11485, 0.06731752),
 (5860, 0.0667864),
 (24141, 0.06470739),
 (14121, 0.063985184),
 (24098, 0.06384958),
 (28864, 0.063740924)]

In [36]:
# bookmarks.dat
# id	md5	title	url	md5Principal	urlPrincipal
# df_bookmarks = pd.read_table('bookmarks.dat')
# хотела вывести названия рекомендованных bookmarks, но
# все танцы с бубном вокруг кодировки и первого символа не увенчались успехом

In [37]:
for r in recommendations50:
    res = df_bookmarks[df_bookmarks['id'] == r[0]]
    print(res['id'].tolist()[0], res['title'].tolist()[0])

NameError: name 'df_bookmarks' is not defined

item-to-item

In [38]:
iid = 2178

In [39]:
items_to_item50 = model50.similar_items(iid)
items_to_item50

[(2178, 1.573555),
 (45334, 1.5663882),
 (45277, 1.5663882),
 (45305, 1.5663882),
 (45294, 1.5663882),
 (45356, 1.5663882),
 (45290, 1.5663882),
 (45357, 1.5663882),
 (45325, 1.5663882),
 (45299, 1.5663881)]

In [ ]:
for i in items_to_item50:
    res = df_bookmarks[df_bookmarks['id'] == i[0]]
    print(res['id'].tolist()[0], res['title'].tolist()[0])